In [1]:
# -*- coding: utf-8 -*-
"""
@author: ancintro
"""

'\n@author: ancintro\n'

In [2]:
# dsnakeyvaulthelper is avaialable in our private package manager, you need to setup a pip.conf. Follow instructions in our Azure DevOps package manager section
from azure.identity import ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
import azure.identity
import os

In [3]:
credential = ManagedIdentityCredential()

#Change this only if you are not using our reasearchlab azure storage account
secret_client = SecretClient(
    vault_url="https://researchlab-wu-kv.vault.azure.net/",
    credential=credential
)
vault_secret_name ='researchlabwusstorage-key-3df4'

In [4]:
#Change this only if you are not using our reasearchlab azure storage account
account_name = 'researchlabwusstorage'
account_key = secret_client.get_secret(vault_secret_name).value

In [5]:
#No need to change this
os.environ["AZURE_STORAGE_ACCOUNT"] = account_name
os.environ["AZURE_STORAGE_ACCESS_KEY"] = account_key

In [6]:
#Change this to the desired values
os.environ["FUSE_MOUNT_PATH"] = '/mnt/akoustos'
os.environ["FUSE_MOUNT_CONTAINER_NAME"] = 'akoustos'

In [7]:
%%bash
sudo mkdir -p /blobfuseconnection
sudo chown $NB_USER /blobfuseconnection/
sudo echo "accountName" $AZURE_STORAGE_ACCOUNT > /blobfuseconnection/connection.cfg
sudo echo "accountKey" $AZURE_STORAGE_ACCESS_KEY >> /blobfuseconnection/connection.cfg
sudo echo "containerName" $FUSE_MOUNT_CONTAINER_NAME >> /blobfuseconnection/connection.cfg

In [8]:
%%bash
sudo mkdir -p $FUSE_MOUNT_PATH
sudo chown $NB_USER $FUSE_MOUNT_PATH
sudo mkdir -p /mnt/resource/
sudo chown $NB_USER /mnt/resource/

In [9]:
%%bash
sudo blobfuse $FUSE_MOUNT_PATH --tmp-path=/mnt/resource/ --container-name=$FUSE_MOUNT_CONTAINER_NAME --config-file=/blobfuseconnection/connection.cfg -o attr_timeout=240 -o entry_timeout=240 -o negative_timeout=120 -o uid=1000 -o gid=100 -o allow_other
sudo rm -f connection.cfg